Train a LSTM on the IMDB sentiment classification task.
-------

![](http://www.clipartbest.com/cliparts/di8/ykA/di8ykA4ie.jpeg)

----
By The End Of This Session You Should Be Able To:
----

- Create and tune a LSTM on real world data
- Describe the limitations of LSTMs

[Based on this code](https://github.com/fchollet/keras/blob/master/examples/imdb_lstm.py)

In [1]:
reset -fs

In [2]:
from keras.datasets import imdb

Using TensorFlow backend.


> Dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".

> As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.

Read more about the data [here](https://keras.io/datasets/) and [here](http://ai.stanford.edu/~amaas/data/sentiment/)

Here is a sample review:
    
> Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!

TODO: Why are the review encoded as a sequence of word indexes and __not__ as strings?

In [4]:
print('Loading data...')
max_features = 20000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


In [6]:
from keras.preprocessing import sequence
print('Pad sequences (samples x time)')

maxlen = 80  # Cut texts after this number of words (among top max_features most common words)

x_train = sequence.pad_sequences(x_train, 
                                 maxlen=maxlen)
x_test = sequence.pad_sequences(x_test,
                                maxlen=maxlen)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


TODO: Why do we need to pad the sequences?

<br>
<details><summary>
Click here for a hint…
</summary>
Before padding, what is the shape of the review #0 and #1?
</details>

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

In [8]:
print('Build model...')
model = Sequential()

max_features = 20000
model.add(Embedding(max_features,
                    128))

model.add(LSTM(128, 
               dropout=0.2, 
               recurrent_dropout=0.2))

model.add(Dense(1, 
                activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

Build model...


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 2,691,713.0
Trainable params: 2,691,713
Non-trainable params: 0.0
_________________________________________________________________


TODO: Summarize this model in 1 or 2 sentences:

TODO: How does the size of the data compare to the number parameters in the model? Is this a problem?

In [11]:
print('Training...')

epochs = 1
batch_size = 32

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

print('Testing...')
score, accuracy = model.evaluate(x_test, y_test,
                                 batch_size=batch_size)

Training...
Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 381s - loss: 0.6928 - acc: 0.5110 - val_loss: 0.6928 - val_acc: 0.5030

In [12]:
print('Test score: {:.3}'.format(score))
print('Test accuracy: {:.3}'.format(accuracy))

Test score: 0.693
Test accuracy: 0.503


TODO: How does the model do? 

TODO: What other machine learning models would fit this data? Which one would you use and why?

TODO: Modify the model anyway you want to improve its performance

Stanford researchers in a 2011 achieved an accuracy of 88.89%. Kaggle competition titled “Bag of Words Meets Bags of Popcorn” in late 2014 to early 2015. Accuracy was achieved above 97% with winners achieving 99%.

Notes:
- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.

In [13]:
from keras.layers import Dense, Dropout

In [14]:
print('Build model...')
model = Sequential()

max_features = 20000
model.add(Embedding(max_features,
                    128))

model.add(LSTM(128, 
               dropout=0.2, 
               recurrent_dropout=0.2, return_sequences=True))

model.add(Dense(128, activation='relu'))
model.add(Dropout(.3))
model.summary()
model.add(LSTM(128, 
               dropout=0.2, 
               recurrent_dropout=0.2))

model.add(Dense(1, 
                activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Training...')

epochs = 1
batch_size = 128

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))



Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 128)         131584    
_________________________________________________________________
dense_2 (Dense)              (None, None, 128)         16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 128)         0         
Total params: 2,708,096.0
Trainable params: 2,708,096.0
Non-trainable params: 0.0
_________________________________________________________________
Training...
Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 487s - loss: 0.4720 - acc: 0.7664 - val_loss: 0.3726 - val_acc: 0.8384

In [15]:
print('Testing...')
score, accuracy = model.evaluate(x_test, y_test,
                                 batch_size=batch_size)

Testing...
25000/25000 [==============================] - 111s     

In [16]:
print('Test score: {:.3}'.format(score))
print('Test accuracy: {:.3}'.format(accuracy))

Test score: 0.373
Test accuracy: 0.838


----
Fun Activity
-----

Train a LSTM to generate words.

1. Start running code (it takes a while)
2. Read through the code
3. Train on your corpus
4. Share your favorite new sentence

Suggested corpus:

- Trump's tweets https://data.world/briangriffey/trump-tweets
- Elon Musk tweets https://data.world/adamhelsinger/elon-musk-tweets-until-4-6-17

Based on [this code](https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py)

In [ ]:
%run lstm_text_generation.py

<br>
<br> 
<br>

----